In [29]:
import wfdb
import os
import pandas as pd
import numpy as np
import scipy.io as sio

In [30]:
root = 'training2017'

In [31]:
with open('RECORDS', 'r') as f:
    heads = f.readlines()
heads = [head.strip() for head in heads]

In [32]:
signals = {}
for head in heads:
    record = wfdb.rdrecord(os.path.join(root, head))
    signal = record.p_signal
    signal = np.array(signal).reshape(-1)
    signals[head] = signal

In [43]:
label_df = pd.read_csv('REFERENCE-v3.csv', header=None, index_col=0)
labels = label_df.to_dict()[1]
len(labels.keys())

8528

In [44]:
data = {}
for k, v in signals.items():
    cnt = len(v) // 3000
    if cnt == 0:
        continue
    data[k] = [v[cnt*3000:(cnt+1)*3000] for cnt in range(cnt)]

8512